In [10]:
import os
import json
from google.cloud import bigquery
import pandas as pd
import dateparser
from history_page import HistoryPage
from profile_page import ProfilePage

In [11]:
def create_dataset(dataset_ref, location='US'):
    dataset = bigquery.Dataset(dataset_ref)
    try:
        dataset = client.create_dataset(dataset)
        print('\n-- Dataset {} created --'.format(dataset_id))
    except:
        print('\n-- Dataset {} already exists --'.format(dataset_id))
        
def check_table_exists(table_ref):
    try:
        client.get_table(table_ref)
        return True
    except:
        return False

In [12]:
def get_schema_from_json(json, schema=[]):
    input_json = json.copy()
    if not input_json:
        return schema

    cur = input_json.pop()
    name = cur['name']
    field_type = cur['type']
    mode = cur['mode']
    fields = [] if 'fields' not in cur else get_schema_from_json(cur['fields'], [])
    schema.append(bigquery.SchemaField(name=name, field_type=field_type, mode=mode, fields=fields))

    return get_schema_from_json(input_json, schema)

In [13]:
def divide_into_batches(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [14]:
def database_setup(configuration, client, dataset_id):
    dataset_ref = client.dataset(dataset_id)
    create_dataset(dataset_ref, location='US')
    
    for table_name in ['northrend', 'azeroth']:
        fields = configuration.get('fields')
        schema = get_schema_from_json(fields, [])
        file_path = './data_backfill/{}.json'.format(table_name)
        try:
            with open(file_path, 'r') as f:
                data = json.loads(f.read())
        except Exception:
            print('No data found')
        
        create_table(dataset_id, table_name, schema, data)

In [15]:
def create_table(dataset_id, table_name, schema, data):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_name)
    table = bigquery.Table(table_ref, schema=schema)
    if check_table_exists(table_ref):
        print('\n-- Table {} already exists --\n'.format(table_ref))
        return
    
    table = client.create_table(table)
    batches = divide_into_batches(data, 10000)
    for batch in batches:
        errors = client.insert_rows(table, batch)
        try:
            assert errors == []
        except Exception:
            print(Exception, errors[0])
    print('\n-- Table {} created --\n'.format(table_name))

In [30]:
def update_table(dataset_id, table_name, new_games):
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_name)
    table = client.get_table(table_ref)
    errors = client.insert_rows(table, new_games)#, insertId='game_id')
    print('Uploaded {} new games'.format(len(new_games)))

In [17]:
bigquery_credpath = os.path.abspath('/Users/cdaly/Box Sync/Daly, Christopher/Keys/BigQuery Reader Project-88493810ca62.json')
client = bigquery.Client.from_service_account_json(bigquery_credpath)
job_config = bigquery.LoadJobConfig()
job_config.skip_leading_rows = 1
job_config.autodetect = True
dataset_id = 'wc3'

with open('./table_config.json', 'r') as f:
    configuration = json.loads(f.read())
    
database_setup(configuration, client, dataset_id)


-- Dataset wc3 already exists --

-- Table TableReference(DatasetReference('bigquery-reader-project', 'wc3'), 'northrend') already exists --


-- Table TableReference(DatasetReference('bigquery-reader-project', 'wc3'), 'azeroth') already exists --



## Dev records

## Get the most recent data for player one

In [18]:
def get_last_bq_date(client, player, server):
    query = ('''SELECT date FROM  `bigquery-reader-project.wc3.{}`,
              UNNEST(team_one) AS first
            WHERE 
                LOWER(first) IN (LOWER('{}'))
            ORDER BY date DESC LIMIT 1
        '''.format(server, player))

    job = client.query(query)
    results = list(job.result())
    if len(results) == 0:
        print('Player ({}) not in table ({}).'.format(player, server))
        return None
    row = results[0]
    last_bq_date = row.get('date')
    print('Last BigQuery date: {}'.format(last_bq_date))
    return last_bq_date

In [19]:
def get_games_between_players(client, players, server):
    query = ('''SELECT * FROM  `bigquery-reader-project.wc3.{}`,
              UNNEST(team_one) AS first,
              UNNEST(team_two) AS second
            WHERE 
                LOWER(first) IN (LOWER('{player_one}'), LOWER('{player_two}')) 
            AND 
                LOWER(second) in (LOWER('{player_one}'), LOWER('{player_two}'))
            ORDER BY date DESC
        '''.format(server, **players))

    job = client.query(query)
    results = job.to_dataframe().to_dict(orient='records')
    if len(results) == 0:
        print('No games between ({player_one}) and ({player_two}) in database.'.format(**players))
    return results

In [20]:
def get_new_games(player, server, last_bq_date=None):
    print('Looking for new games...')
    def no_more_dates():
        if last_bq_date is None:
            return False
        else:
            return current_date <= last_bq_date
        
    def no_more_pages():
        return page >= next_page
    
    new_games = []
    page = 1
    
    while True:
        
        history_page = HistoryPage(player, server, page)
        games = history_page.games
        next_page = history_page.next_page

        for d in games:
            current_date = dateparser.parse(d.get('date'))
            if no_more_dates():
                return new_games
            else:
                new_games.append(d)
        
        if no_more_pages():
            return new_games
        else:
            page = next_page

In [21]:
def calculate_record(data_input, games):
    df = pd.DataFrame(games)
    
    record = {}
    for player in data_input.get('players').values():
        record[player] = df['winner'].apply(lambda x: player.lower() == x[0].lower()).sum()
    
    return record

In [22]:
def lambda_handler(data_input):
    bigquery_credpath = os.path.abspath('/Users/cdaly/Box Sync/Daly, Christopher/Keys/BigQuery Reader Project-88493810ca62.json')
    client = bigquery.Client.from_service_account_json(bigquery_credpath)
    job_config = bigquery.LoadJobConfig()
    job_config.skip_leading_rows = 1
    job_config.autodetect = True
    dataset_id = 'wc3'
    
    player = data_input.get('players').get('player_one')
    server = data_input.get('server')

    # Check how recent the table data is for player_one. 
    last_bq_date = get_last_bq_date(client, player, server)
    
    # Update current data.
    try:
        new_games = get_new_games(player, server, last_bq_date=last_bq_date)
        if len(new_games) > 0:
            update_table(dataset_id, server, new_games)
        else:
            print('No new games.')
    except Exception:
        print('Bnet is not responding. Not updating tables.')
        print(Exception)
        
    # Get current record.
    games = get_games_between_players(client, data_input.get('players'), server)
    if len(games) > 0:
        record = calculate_record(data_input, games)
        data = {'games': games, 'record': record}
    else:
        data = None
    return data

In [23]:
data_input = {
    'server': 'northrend',
    'players': {
        'player_one': 'followgrubby',
        'player_two': 'romantichuman',
    }
}
player = data_input.get('players').get('player_one')
server = data_input.get('server')

lambda_handler(data_input)

Last BigQuery date: 2018-12-04 02:07:00
Looking for new games...
Bnet is not responding. Not updating tables.
<class 'Exception'>


{'games': [{'date': Timestamp('2018-11-24 23:36:00'),
   'first': 'RomanticHuman',
   'game_id': 79729629,
   'game_length': 0,
   'game_type': 'Solo',
   'map': 'Echo Isles',
   'second': 'FollowGrubby',
   'team_one': ['RomanticHuman'],
   'team_two': ['FollowGrubby'],
   'winner': ['FollowGrubby']},
  {'date': Timestamp('2018-11-24 23:36:00'),
   'first': 'FollowGrubby',
   'game_id': 79729629,
   'game_length': 0,
   'game_type': 'Solo',
   'map': 'Echo Isles',
   'second': 'RomanticHuman',
   'team_one': ['FollowGrubby'],
   'team_two': ['RomanticHuman'],
   'winner': ['FollowGrubby']},
  {'date': Timestamp('2018-11-24 23:36:00'),
   'first': 'RomanticHuman',
   'game_id': 79729629,
   'game_length': 0,
   'game_type': 'Solo',
   'map': 'Echo Isles',
   'second': 'FollowGrubby',
   'team_one': ['RomanticHuman'],
   'team_two': ['FollowGrubby'],
   'winner': ['FollowGrubby']},
  {'date': Timestamp('2018-11-24 23:36:00'),
   'first': 'FollowGrubby',
   'game_id': 79729629,
   'game_

In [24]:
last_bq_date = get_last_bq_date(client, player, server)
last_bq_date

Last BigQuery date: 2018-12-04 02:07:00


datetime.datetime(2018, 12, 4, 2, 7)

In [25]:
new_games = get_new_games(player, server, last_bq_date=last_bq_date)

Looking for new games...


In [40]:
new_games = get_new_games(player, server)
print(len(new_games))

Looking for new games...
40


In [31]:
pd.DataFrame(new_games)

,date,game_id,game_length,game_type,map,team_one,team_two,winner
0,2019-02-06 00:56:00,79862681,21,Solo,Terenas Stand_ L V,[followgrubby],[JUSTdo[it]],[followgrubby]
1,2019-02-03 01:22:00,79845695,11,Solo,Twisted Meadows,[followgrubby],[pwarrior],[pwarrior]
2,2019-02-03 01:08:00,79845644,12,Solo,Concealed Hill,[followgrubby],[Uberelf],[followgrubby]
3,2019-02-03 00:52:00,79845577,12,Solo,Last Refuge,[followgrubby],[atm],[followgrubby]
4,2019-02-03 00:37:00,79845522,9,Solo,Northern Isles,[followgrubby],[Critical],[followgrubby]
5,2019-02-02 22:57:00,79844958,10,Solo,Twisted Meadows,[followgrubby],[Alchh],[followgrubby]
6,2019-02-02 22:40:00,79844851,6,Solo,Turtle Rock,[followgrubby],[koza],[followgrubby]
7,2019-02-02 22:30:00,79844780,13,Solo,Terenas Stand_ L V,[followgrubby],[IamTry],[followgrubby]
8,2019-02-02 22:15:00,79844681,11,Solo,Twisted Meadows,[followgrubby],[tonbegali],[followgrubby]
9,2019-02-02 20:06:00,79843923,19,Solo,Echo Isles,[followgrubby],[FoxyDi],[followgrubby]


In [28]:
last_bq_date

datetime.datetime(2018, 12, 4, 2, 7)

In [32]:
if len(new_games) > 0:
    update_table(dataset_id, server, new_games)

Uploaded 10 new games
